# Metrics manipulattion

Key resources
* https://docs.ragas.io/en/stable/concepts/metrics/index.html
* https://github.com/explodinggradients/ragas/tree/main/src/ragas/metrics (prompts and logic)
* https://blog.langchain.dev/evaluating-rag-pipelines-with-ragas-langsmith/ (see the 'under the hood' section)

## prep

In [1]:
# !pip install -U openai

In [2]:
# !pip install openai
# # !pip uninstall -y openai

In [3]:
# import openai
# openai.__version__

In [4]:
# !pip install ragas

In [1]:
from copy import deepcopy
import os
import openai

In [11]:
from dotenv import load_dotenv
load_dotenv()

openaikey = os.getenv('OPENAI_API_KEY')
openai.api_key = openaikey

## Alter inputs to understand metrics
* https://blog.langchain.dev/evaluating-rag-pipelines-with-ragas-langsmith/

## Eval single doc

In [1]:
import langchain

In [3]:
from langchain.document_loaders import WebBaseLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

In [13]:
## set up and run the retrieval chain.
# - for one doc chunked.

# load the Wikipedia page and create index
loader = WebBaseLoader("https://en.wikipedia.org/wiki/New_York_City")
index = VectorstoreIndexCreator().from_loaders([loader])

# create the QA chain
llm = ChatOpenAI()
qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=index.vectorstore.as_retriever(), return_source_documents=True
)

# testing it out
question = "How did New York City get its name?"
result = qa_chain({"query": question})
result["result"]

'New York City was named after the Duke of York, who later became King James II of England. In 1664, King Charles II appointed his brother, the Duke of York, as the proprietor of the territory of New Netherland, which included the city of New Amsterdam. When England seized control of the area from the Dutch, it was renamed New York in honor of the Duke.'

In [16]:
result.keys()

dict_keys(['query', 'result', 'source_documents'])

In [17]:
results_orig = deepcopy(result)

In [18]:
## Prep the eval chain.
#
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall
from ragas.langchain import RagasEvaluatorChain

# make eval chains
eval_chains = {m.name: RagasEvaluatorChain(metric=m) for m in [faithfulness, answer_relevancy, context_precision] } # context_recall

In [20]:
# Run each eval separately.
for name, eval_chain in eval_chains.items():
    score_name = f"{name}_score"
    print(f"{score_name}: {eval_chain(result)[score_name]}")

faithfulness_score: 1.0
answer_relevancy_score: 0.9329907247343107
context_precision_score: 0.99999999995


## faithfullness (answer to the context)
* answer is regarded as faithful if all the claims that are made in the answer can be inferred from the given context.
* One prompt to extract statements (sentences?) and a second prompt to judge groundedness.

In [21]:
RagasEvaluatorChain(metric=faithfulness)(result)['faithfulness_score']

1.0

In [24]:
result = deepcopy(results_orig)

In [ ]:
## Adding eroneous contexts/docs should NOT affect faithfulness.
# - sonce all the answers are still available in the context docs.

In [ ]:
# Add four bad docs to the end of the results list.
myd = deepcopy(myr['source_documents'][0])
myd.page_content = 'dog '*50
result['source_documents'].extend([myd, myd, myd, myd])
result['source_documents']

[Document(page_content="Etymology\nSee also: Nicknames of New York City\nIn 1664, New York was named in honor of the Duke of York (later King James\xa0II of England).[35] James's elder brother, King Charles\xa0II, appointed the Duke as proprietor of the former territory of New Netherland, including the city of New Amsterdam, when England seized it from Dutch control.[36]", metadata={'source': 'https://en.wikipedia.org/wiki/New_York_City', 'title': 'New York City - Wikipedia', 'language': 'en'}),
 Document(page_content='New York City traces its origins to Fort Amsterdam and a trading post founded on the southern tip of Manhattan Island by Dutch colonists in approximately 1624. The settlement was named New Amsterdam (Dutch: Nieuw Amsterdam) in 1626 and was chartered as a city in 1653. The city came under British control in 1664 and was renamed New York after King Charles II granted the lands to his brother, the Duke of York.[21] The city was temporarily regained by the Dutch in July 1673

In [ ]:
# This should have no effect on faithfulness, it's just extra, unused context.
RagasEvaluatorChain(metric=faithfulness)(result)['faithfulness_score']

1.0

In [ ]:
## removing all the relevant docs should kill faithfulness.
#
result['source_documents'] = result['source_documents'][4:]
result['source_documents']

[Document(page_content='dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog ', metadata={'source': 'https://en.wikipedia.org/wiki/New_York_City', 'title': 'New York City - Wikipedia', 'language': 'en'}),
 Document(page_content='dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog ', metadata={'source': 'https://en.wikipedia.org/wiki/New_York_City', 'title': 'New York City - Wikipedia', 'language': 'en'}),
 Document(page_content='dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog ', metadata={'source': 'https://en.wikipedia.org/wiki/New_York_City', 't

In [ ]:
RagasEvaluatorChain(metric=faithfulness)(result)['faithfulness_score']

0.0

## answer_relevancy
* measures the relevancy of the answer to the question.
* gens questions from answer then measures similarity of these generate questions to original question.
* does not seem to work as expected. When kill answer and context-docs still getting 0.7 score.

In [25]:
result = deepcopy(results_orig)

In [26]:
RagasEvaluatorChain(metric=answer_relevancy)(result)['answer_relevancy_score']

0.9329907247343107

In [28]:
result.keys()

dict_keys(['query', 'result', 'source_documents'])

In [27]:
# result['result']
result

{'query': 'How did New York City get its name?',
 'result': 'New York City was named after the Duke of York, who later became King James II of England. In 1664, King Charles II appointed his brother, the Duke of York, as the proprietor of the territory of New Netherland, which included the city of New Amsterdam. When England seized control of the area from the Dutch, it was renamed New York in honor of the Duke.',
 'source_documents': [Document(page_content="Etymology\nSee also: Nicknames of New York City\nIn 1664, New York was named in honor of the Duke of York (later King James\xa0II of England).[35] James's elder brother, King Charles\xa0II, appointed the Duke as proprietor of the former territory of New Netherland, including the city of New Amsterdam, when England seized it from Dutch control.[36]", metadata={'language': 'en', 'source': 'https://en.wikipedia.org/wiki/New_York_City', 'title': 'New York City - Wikipedia'}),
  Document(page_content="Etymology\nSee also: Nicknames of N

In [ ]:
# result['result']  = result['result'] *4
# result['result']
# RagasEvaluatorChain(metric=answer_relevancy)(result)['answer_relevancy_score']

In [29]:
# Should kill this metric.
result['result'] = 'dog ' * 50
result['result'] 

'dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog '

In [31]:
# Seems like this should be much much lower.
RagasEvaluatorChain(metric=answer_relevancy)(result) # ['answer_relevancy_score']

{'query': 'How did New York City get its name?',
 'result': 'dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog ',
 'source_documents': [Document(page_content="Etymology\nSee also: Nicknames of New York City\nIn 1664, New York was named in honor of the Duke of York (later King James\xa0II of England).[35] James's elder brother, King Charles\xa0II, appointed the Duke as proprietor of the former territory of New Netherland, including the city of New Amsterdam, when England seized it from Dutch control.[36]", metadata={'language': 'en', 'source': 'https://en.wikipedia.org/wiki/New_York_City', 'title': 'New York City - Wikipedia'}),
  Document(page_content="Etymology\nSee also: Nicknames of New York City\nIn 1664, New York was named in honor of the Duke of York (later King James\xa0II of England).[35] James's elder brother, King Charles\xa0II,

In [33]:
## Alter the context to see what that does now that answer way wrong.
# - the prompt for this measure uses the context and the answer to generate the question.

# Add four bad docs to the end of the results list.
myd = deepcopy(result['source_documents'][0])
myd.page_content = 'dog '*50
result['source_documents'] = [myd, myd, myd, myd]
result['source_documents']

[Document(page_content='dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog ', metadata={'language': 'en', 'source': 'https://en.wikipedia.org/wiki/New_York_City', 'title': 'New York City - Wikipedia'}),
 Document(page_content='dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog ', metadata={'language': 'en', 'source': 'https://en.wikipedia.org/wiki/New_York_City', 'title': 'New York City - Wikipedia'}),
 Document(page_content='dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog ', metadata={'language': 'en', 'source': 'https://en.wikipedia.org/wiki/

In [36]:
RagasEvaluatorChain(metric=answer_relevancy)(result) # ['answer_relevancy_score']

{'query': 'How did New York City get its name?',
 'result': 'dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog ',
 'source_documents': [Document(page_content='dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog ', metadata={'language': 'en', 'source': 'https://en.wikipedia.org/wiki/New_York_City', 'title': 'New York City - Wikipedia'}),
  Document(page_content='dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog ', metadata={'language': 'en', 'source': 'https://en.wikipedia.org/wiki/New_York_City', 'title': 'New York City - Wikipedia'}),
  Document

## context precision.
* https://docs.ragas.io/en/stable/concepts/metrics/context_precision.html
* Search results order is key to this metric. Must have the answers in the top few context chunks to get a good score.
* So this retreival metric is all about returning good well ranked results.

In [ ]:
result = deepcopy(results_orig)

In [38]:
RagasEvaluatorChain(metric=context_precision)(result) # ['context_precision_score']

{'query': 'How did New York City get its name?',
 'result': 'dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog ',
 'source_documents': [Document(page_content='dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog ', metadata={'language': 'en', 'source': 'https://en.wikipedia.org/wiki/New_York_City', 'title': 'New York City - Wikipedia'}),
  Document(page_content='dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog ', metadata={'language': 'en', 'source': 'https://en.wikipedia.org/wiki/New_York_City', 'title': 'New York City - Wikipedia'}),
  Document

In [ ]:
# Add bad docs at end of results list. Should have little or no effect on context precision.
# - this metric is all about search results rank.

myd = deepcopy(result['source_documents'][0])
myd.page_content = 'dog '*50
result['source_documents'].extend([myd, myd, myd, myd])

In [ ]:
result['source_documents']

[Document(page_content="Etymology\nSee also: Nicknames of New York City\nIn 1664, New York was named in honor of the Duke of York (later King James\xa0II of England).[35] James's elder brother, King Charles\xa0II, appointed the Duke as proprietor of the former territory of New Netherland, including the city of New Amsterdam, when England seized it from Dutch control.[36]", metadata={'source': 'https://en.wikipedia.org/wiki/New_York_City', 'title': 'New York City - Wikipedia', 'language': 'en'}),
 Document(page_content='New York City traces its origins to Fort Amsterdam and a trading post founded on the southern tip of Manhattan Island by Dutch colonists in approximately 1624. The settlement was named New Amsterdam (Dutch: Nieuw Amsterdam) in 1626 and was chartered as a city in 1653. The city came under British control in 1664 and was renamed New York after King Charles II granted the lands to his brother, the Duke of York.[21] The city was temporarily regained by the Dutch in July 1673

In [ ]:
RagasEvaluatorChain(metric=context_precision)(result)['context_precision_score']

0.99999999995

In [ ]:
## reverse this list.
# - this should kill this metric.

result['source_documents'] = result['source_documents'][::-1]

In [ ]:
result['source_documents']

[Document(page_content='dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog ', metadata={'source': 'https://en.wikipedia.org/wiki/New_York_City', 'title': 'New York City - Wikipedia', 'language': 'en'}),
 Document(page_content='dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog ', metadata={'source': 'https://en.wikipedia.org/wiki/New_York_City', 'title': 'New York City - Wikipedia', 'language': 'en'}),
 Document(page_content='dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog ', metadata={'source': 'https://en.wikipedia.org/wiki/New_York_City', 't

In [ ]:
# RagasEvaluatorChain(metric=context_relevancy)(result)['context_relevancy_score']

0.19642857141875